In [45]:
import numpy as np
import matplotlib.pyplot as plt
from moarchiving import BiobjectiveNondominatedSortedList as NDA
import cma
from comocma import Sofomore, get_cmas, FitFun
from importlib import reload
from pathlib import Path
import rebase
from itertools import cycle
import bidict

reload(rebase)
from rebase import SofomorePatch, kernelBuilder, Multistart, RestartSquencer

In [46]:
other_attributes = {"incumbent": property(lambda self: self.mean)}
Kernel = kernelBuilder(cma.CMAEvolutionStrategy, other_attributes=other_attributes)

In [74]:
dim = 2
sigma0 = 2.0
budget = 600  # * int(4 + 3 * np.log(dim))

cma_options = [cma.CMAOptions(), cma.CMAOptions()]
for irestart, cma_option in enumerate(cma_options, start=1):
    cma_option["verbose"] = -1
    cma_option["maxiter"] = 30 * irestart
sofomore_options = {
    "reference_point": [16, 16],  # For hypervolume computation only not for the UHVI
    "opts": {"verb_filename": "sofomore_out_1"},
}

# -----------------------------------------------------------
num_of_starts = 1
num_of_restarts = [3, 3]
# num_of_restarts = [59]
start_options = [  # list[Restarts][number_of_starts][number_of_kernels]
    [
        [
            {
                "x0": x,
                "sigma0": sigma0 * (0.2) ** irestart,
                "inopts": cma_option,
            }
            for k, x in enumerate(np.random.rand(num_kernels, dim))
        ]
        for i in range(num_of_starts)
    ]
    for irestart, (num_kernels, cma_option) in enumerate(
        zip(num_of_restarts, cma_options)
    )
]
# --------------------------------------------------------------------
run = RestartSquencer(start_options, Kernel, budget, sofomore_options)

In [75]:
fun = FitFun(
    lambda x: np.sqrt(cma.ff.sphere(x)),
    lambda x: np.sqrt(cma.ff.sphere(x - np.asarray([16, 0]))),
)

In [63]:
n = 8
b = 550
p = (b - n) // n
cb = b
used = 0
for i in range(n):
    cb -= p + 1
    print(f"{cb = }")

    if cb < (p + 1):
        p += cb

    used += p + 1
    print(f"{p = }")

print(f"{used=}")

cb = 482
p = 67
cb = 414
p = 67
cb = 346
p = 67
cb = 278
p = 67
cb = 210
p = 67
cb = 142
p = 67
cb = 74
p = 67
cb = 6
p = 73
used=550


In [76]:
old_hypervolume = 0
tols = []
hist_len = 3
tol_thresh = 1e-3
hypervolume_threshold = False
run.archive = bidict.bidict()
hv_max = 0

for irestart, moes in enumerate(run):
    print(f"Restart {irestart}")
    sub_archive = bidict.bidict()
    flag = True

    print(*[k.popsize for k in moes], sep=", ")
    while not moes.stop():
        # if flag:
        #     print(f"Budget = {len(solutions)}")
        #     print(f"number of active kernels = {len(moes._active_indices)}")
        #     flag = False
        solutions = moes.ask("all")
        objective_values = [fun(solution) for solution in solutions]
        moes.tell(solutions, objective_values)
        moes.logger.add()
        moes.disp()

        # ---------------------------------------------------------------------------
        try:
            sub_archive.update(
                {
                    tuple(np.asarray(s).tolist()): tuple(np.asarray(f).tolist())
                    for s, f in zip(solutions, objective_values)
                }
            )
        except:
            pass
        # print(archive)

        EPF = [mo.objective_values for mo in moes if mo is not None]

        if len(EPF) == 0:
            continue

        current_hypervolume = NDA(EPF, moes.reference_point).hypervolume
        if current_hypervolume > hv_max:
            hv_max = current_hypervolume
        tol = float(abs(current_hypervolume - old_hypervolume) / hv_max)
        old_hypervolume = current_hypervolume
        tols.append(tol)

        hypervolume_threshold = np.all((np.asarray(tols) < tol_thresh)[-hist_len:])

        # for kernel in moes:
        #     kernel._stopdict._addstop("hvtol", hypervolume_threshold, tol)

    for fval in moes.archive:
        value = tuple(np.asarray(fval).tolist())

        if value in sub_archive.inverse:
            run.archive.update({sub_archive.inverse[value]: value})

    tol_thresh *= (1 / len(moes)) * (1e-5)

Restart 0
199, 199, 199
Iterat #Fevals   Hypervolume   axis ratios   sigmas   min&max stds
                                  (median)  (median)    (median)
    1    600 1.354832228142391e+01 1.0e+00 4.45e+00  3e+00  7e+00
    2   1200 4.661086213669302e+01 1.8e+00 7.11e+00  4e+00  1e+01
    3   1800 7.460164177481599e+01 2.6e+00 7.06e+00  2e+00  5e+00
   30  18000 9.599999743469834e+01 2.3e+00 1.94e+03  5e-04  9e-04
Restart 1
199, 199, 199, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49
   90  54003 1.199999990787902e+02 2.3e+00 3.33e+00  8e-09  3e-08


In [30]:
# f, ax = plt.subplots()

# X = np.asarray(archive.values())

# # ax.scatter(X, Y)
# print(X[:10])

# k = dict(archive).keys()

mid = tuple(moes.archive[len(moes.archive) // 2])
archive.inverse[mid]

(28.959817797511384, -0.028299006213052705)

In [278]:
from itertools import cycle

f, ax = plt.subplots()
ax.minorticks_on()
ax.grid(True)
ax.grid(which="minor", alpha=0.5)
ax.set_xlim(-5, 65)
ax.set_ylim(-5, 65)
ax.set_aspect(1)

visited = []
marker = cycle((",", "+", ".", "o"))

ymin_val, ymax_val = ax.get_ylim()
xmin_val, xmax_val = ax.get_xlim()

ratio_x = lambda x: (x - xmin_val) / (xmax_val - xmin_val)  # noqa: E731
ratio_y = lambda x: (x - ymin_val) / (ymax_val - ymin_val)  # noqa: E731

numR = len(num_of_restarts)
restarts = [[] for _ in range(numR)]

for kernel in moes[:]:
    restarts[kernel.irestart - 1].append(kernel)

for irestart, restart in enumerate(restarts):
    for kernel in restart:
        color = None

        if kernel not in visited:
            mrk = next(marker)
            rx, ry = kernel.reference_point
            # print(rx, ry)
            sc = ax.scatter(
                [rx],
                [ry],
                marker="+",
            )
            color = sc.get_facecolor()
            ax.annotate(
                f"ref: ({rx:.1f}, {ry:.1f})",
                (rx, ry),
                textcoords="offset points",
                xytext=(10, 10),
                ha="left",
                fontsize=7,
            )
            ax.axvline(
                x=rx,
                ymax=ratio_y(ry),
                color=color,
                alpha=0.8,
                ls="--",
                lw=1.0,
            )
            ax.axhline(
                y=ry,
                xmax=ratio_x(rx),
                color=color,
                alpha=0.8,
                ls="--",
                lw=1.0,
            )

        for subkernel in kernel.group_:
            if subkernel not in visited:
                visited.append(subkernel)
                x, y = subkernel.objective_values
                sc = ax.scatter(
                    [x],
                    [y],
                    color=color,
                    marker="x",
                    s=(numR - irestart) ** 2 * 25,
                    lw=0.5 * (numR - irestart),
                )

In [14]:
X, Y = np.asarray(moes.archive).T
# f, ax = plt.subplots()
# ax.scatter(X, Y)

ItemsView(<cma.evolution_strategy._CMASolutionDict_functional object at 0x000001C2000C6C10>)

In [2]:
from bidict import bidict

In [73]:
color = None
restarts = run.restarts_kernels
marker = cycle((",", "+", ".", "o"))
f, ax = plt.subplots()
ax.minorticks_on()
ax.grid(True)
ax.grid(which="minor", alpha=0.5)
ax.set_xlim(-5, 18)
ax.set_ylim(-5, 18)
ax.set_aspect(1)

ymin_val, ymax_val = ax.get_ylim()
xmin_val, xmax_val = ax.get_xlim()

ratio_x = lambda x: (x - xmin_val) / (xmax_val - xmin_val)  # noqa: E731
ratio_y = lambda x: (x - ymin_val) / (ymax_val - ymin_val)  # noqa: E731

for irestart, restart in enumerate(restarts):
    mrk = next(marker)
    for ref_group in restart:
        rx, ry = ref_group[0][0].reference_point

        sc = ax.scatter(
            [rx],
            [ry],
            marker="+",
        )
        color = sc.get_facecolor()

        # X, Y = np.asarray(NDA(moes.archive, reference_point=[rx, ry])).T
        # ax.scatter(X, Y, color=color, marker="+", lw=0.5, alpha=0.1)
        # middle = len(X) // 2

        # ax.scatter([X[middle]], [Y[middle]], color="black", s=100, marker="o")
        ax.annotate(
            f"ref: ({rx:.3f}, {ry:.3f})",
            (rx, ry),
            textcoords="offset points",
            xytext=(10, 10),
            ha="left",
            fontsize=7,
        )
        ax.axvline(
            x=rx,
            ymax=ratio_y(ry),
            color=color,
            alpha=0.8,
            ls="--",
            lw=1.0,
        )
        ax.axhline(
            y=ry,
            xmax=ratio_x(rx),
            color=color,
            alpha=0.8,
            ls="--",
            lw=1.0,
        )
        for start in ref_group:
            for kernel in start:
                x0, y0 = fun(kernel.x0)
                x, y = kernel.objective_values
                ax.scatter([x0], [y0], color="black")
                sc = ax.scatter(
                    [x],
                    [y],
                    color=color,
                    marker=mrk,
                    s=80,
                    # marker="x",
                    # s=(numR - irestart) ** 2 * 25,
                    # lw=0.5 * (numR - irestart),
                )

In [44]:
[k.stop() for k in moes]

[{'hvtol': 3.4186872930075617e-05},
 {'hvtol': 3.4186872930075617e-05},
 {'hvtol': 3.4186872930075617e-05},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40},
 {'maxiter': 40}]

In [72]:
data_hv_1 = moes.logger.load("sofomore_out_1/hypervolume.dat")

iterations, fevals, [hv] = data_hv_1


# f, ax = plt.subplots()
max_hv = max(hv)
ax.plot(iterations, max_hv - np.asarray(hv))
ax.set_xlabel("Iterations")
ax.set_ylabel("Hypervolume")
ax.grid(True)
ax.set_yscale("log")

In [9]:
%matplotlib qt

In [ ]:
max_budget = 1000  # evaluations
max_parallel_budget = 100  # parallel evaluations
min_popsize = 10  # population size

max_number_of_kernels = max_parallel_budget // min_popsize
min_number_of_kernels = 1

number_of_restarts = 2


def restart_counter(previous, num_kernels, max_number_of_kernels):
    previous = sum([count_ for _, count_, _, _ in previous])
    n = int((previous + 1) * num_kernels)

    if n > max_number_of_kernels:
        m = int(max_number_of_kernels // (previous + 1))
        print(m)
        kernel_num = int((previous + 1) * m)
        print(kernel_num)
        popsize = max_parallel_budget // kernel_num
        return (m, kernel_num, popsize, popsize * kernel_num)
    else:
        popsize = max_parallel_budget // num_kernels
        return (num_kernels, n, popsize, popsize * n)


restarts = [(1, 1, max_parallel_budget, max_parallel_budget)]

while number_of_restarts > 0:
    number_of_restarts -= 1
    num_kernels = restart_counter(
        restarts, max_number_of_kernels, max_number_of_kernels
    )
    restarts += [num_kernels]

restarts

In [ ]:
from functools import reduce
from operator import mul


def mult(x):
    if x == []:
        return 1
    return reduce(mul, x)

In [ ]:
max_parallel_budget = 1000  # parallel evaluations
min_popsize = 10  # population size

target_psize = 100

max_psize = max_parallel_budget // min_popsize
min_psize = 1

print(max_psize, min_psize)

k = [1, 6, 7]
lam = [10, 2, 3]


def restart_config(k, lam):
    psize = [sum([mult(k[i : j + 1]) for i in range(j + 1)]) for j in range(len(k))]
    cum_psize = np.cumsum(psize).tolist()
    evals = [l * p for l, p in zip(lam, psize)]

    return [
        psize,
        cum_psize,
        evals,
    ]

In [ ]:
k = [1, 5, 10]
lam = [10, 2, 3]
restart_config(k, lam)

### Tree structure for the restarts

> - The root is the first run then each parent node should produce its value + 1 child nodes  
> - Restart configs will be produced by going down the tree

#### Constraints:
1. Minimum number of kernels at the end $p$
2. Number of parallel evaluations $n$
3. Minimum population size $\lambda_{\text{min}}$

We can combine 2 and 3 by defining n as multiple of $\lambda_{\text{min}}$
and the popsize as a multiple of $\lambda_{\text{min}}$ between 1 and $\lfloor\frac{n}{\lambda_{\text{min}}}\rfloor$

In [ ]:
target = 20

runs = [[100], [10] * 5 * 2, [8] * 1 * 12]
bud = [sum(run) for run in runs]
bud

### Restart Module

- Budget management
- Configuration
- Run optimization

In [ ]:
user_input = dict(
    max_parallel_budget=1000,
    default_popsize=10,
    number_of_kernels=10,
    max_number_of_restarts=4,
    max_number_of_iterations=100,
)

user_input

In [ ]:
refs = np.linspace(0, 1, 4)[1:-1].tolist()
refs_groups = [refs.copy()]

refinement_number = [2, 10, 5][::]

for i in range(len(refinement_number)):
    _refs = [0, *refs, 1]
    j = refinement_number[i]
    current_refs = [
        np.linspace(r1, r2, j - np.random.randint(0, int(j * 0.9)) + 2)[1:-1].tolist()
        for r1, r2 in zip(_refs[:-1], _refs[1:])
    ]
    refs = np.sort(np.hstack((refs, *current_refs)))
    refs_groups.append(current_refs.copy())

len(refs)

In [ ]:
def flatten(sequence):
    flattened = []
    for item in sequence:
        if isinstance(item, list):
            flattened.extend(flatten(item))  # Recursive call if item is a list
        else:
            flattened.append(item)  # Append if item is a float or other non-list type
    return flattened

In [ ]:
%matplotlib qt

In [ ]:
f, ax = plt.subplots()

for i, refs in enumerate(refs_groups):
    # refs = np.asarray(refs).flatten()
    refs = flatten(refs)
    sc = ax.scatter(refs, i * np.ones(len(refs)), label=f"Group {i + 1}", marker="x")
    color = sc.get_facecolor()
    for coord in refs:
        ax.axvline(x=coord, color=color, alpha=0.3, lw=0.8)

ax.legend()
ax.set_xlabel("Reference point")
ax.grid()

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

In [ ]:
points = flatten(refs_groups)

In [ ]:
diff = np.diff(sorted(points))

diff_ = moving_average(diff, 20)
plt.plot(np.linspace(0, 1, len(diff_)), diff_)
plt.grid()

In [ ]:
f, ax = plt.subplots()
ax.boxplot(diff_)
ax.grid()

In [ ]:
%matplotlib qt

In [ ]:
# 1, 2, 3, 5
# lamb: 1, 100
# 1: 100 * 1 = 100 * 1
# 2:  25 * (1 + 1) * 2 = 25 * 2 * 2
# 3:   5 * (1 + 4 + 1 ) * 3 = 5 * 6 * 3
# 5:     * (1 + 4 + 18 + 1) * 5 =   * 24 * 5

In [ ]:
p = 100  # minimum size of the Pareto set
c = 50  # the parallel budget expressed in factor of the default popsize
z = p // c  # the minimum number of restarts

In [ ]:
k = []
n = []

k_ = 70
n_ = c // k_

if n_ == 0:
    k.append([k_ // z] * z)
    n.append([c // k__ for k__ in k[-1]])

k, n

In [ ]:
# k1 + k2 + k3 + k1k2k3 + k3k2 + k1k2 = 70
# k1 = 1 => 2k2 (1 + k3) + k3 = 70
# k2 = 20 => k3(1 + 2k2) = 70 - 2k2
# k3 = (70 - 2k2) / (1 + 2k2)
# dividers of 70: 1, 2, 5, 7, 14, 35, 70
k1 = 2
R = 70
k2 = [i for i in range(1, R // (k1 + 1))]
d = [(k1, i, (R - (k1 + 1) * i) // ((k1 + 1) * i + 1)) for i in k2]
d = [(k1, i, j, (k1 + 1) * i, ((k1 + 1) * i + k1 + 1) * j) for _, i, j in d]
d = [(k1, i, j, m, n, 1 + m + n) for _, i, j, m, n in d]
d = sorted(d, key=lambda x: x[-1], reverse=True)
# d = [(1, i, j, j + i * j) for i, j in d]
# d = [(i, j, m) for i, j, m in d if m <= 50]
# x, y, z = max(d, key=lambda x: x[2])
# u = [i + j + i * j for i, j in d]

# v = [(i, j) for i, j in d if i + j + i * j >= max(u)]

# x, y, z,
d

In [ ]:
k1 = range(1, 6)
k2 = range(1, 11)
# k3 = range(1, 5)
# k4 = range(1, 5)


def pi(k):
    m = []
    m_ = []
    correction = []
    for i, s in enumerate(k):
        runk = (sum(m) + 1) * s
        runk_ = (sum(m_) + 1) * s
        correction += [0]

        while runk_ / runk < 0.95 and correction[-1] + s < 10:
            runk_ = (sum(m_) + 1) * (s + correction[-1])
            correction[-1] += 1

        runk_ = runk_ - np.random.randint(max(1, int(runk_ * 0.4)))
        m += [runk]
        m_ += [runk_]
    return m, m_, correction
    # return k1 + (k1 + 1) * k2 + ((k1 + 1) * (k2 + 1) + 1) * k3


p = []

for u in k1:
    for v in k2:
        # for w in k3:
        #     for z in k4:
        perfect_runs, runs, corr = pi([u, v])  # , w, z])
        factors = [r // k for r, k in zip(runs, (u, v))]  # , w, z))]
        budget = lcm(*runs)
        popsize_factor = [budget // r for r in runs]
        p += [
            (
                u,
                v,
                # w,
                # z,
                perfect_runs,
                runs,
                corr,
                #    budget,
                #    popsize_factor,
                # factors,
                sum(runs),
                sum(perfect_runs),
                f"{100 * sum(runs) / sum(perfect_runs):.2f}%",
            )
        ]

In [ ]:
sorted(p, key=lambda x: x[-2], reverse=True)
# jlist = list(set([m[-1] for m in p]))
# f, ax = plt.subplots()
# ax.scatter(jlist, np.zeros_like(jlist), marker="x")
# f.show()

In [ ]:
from math import lcm

In [ ]:
%matplotlib qt

In [ ]:
r = np.stack([a, b], axis=-1)
q = np.stack([c, d], axis=-1)

N = np.stack([r, q], axis=-2)

# i and j : grid indices
# alpha : u_wind in degrees
# beta : u_current in degrees

a = 10 * np.ones((3, 3))  # sin(alpha), a_tij
b = 100 * np.ones((3, 3))  # sin(beta), b_tij
c = 1000 * np.ones((3, 3))  # cos(alpha), c_tij
d = 10000 * np.ones((3, 3))  # cos(beta), d_tij

r = np.stack([a, b], axis=-1)
q = np.stack([c, d], axis=-1)
N = np.stack([r, q], axis=-2)  # (a_tij, b_tij, c_tij, d_tij)

M = np.arange(9).reshape(3, 3, 1) * np.ones((3, 3, 2))  # (x_ij, y_ij)
U = np.einsum("ijk,ijlk->ijk", M, N)

#                       sin(alpha_t_ij)  cos(alpha_t_ij)
# (x_t_i_j, y_t_i_j)
#                       sin(beta_t_ij)  cos(beta_t_ij)
#

#
#                   a_tij   c_tij
# (x_tij, y_tij)
#                   b_tij   d_tij
#
#
#
# u_k = x_tij1k * a_tijlk
#
# u_k = x_tij1k * a_tijlk + y_tij1k * b_tijlk
#
# delta_t = .1

# x = 0.05 * v_wind
# y = v_current

with np.printoptions(precision=0, suppress=True):
    # print(M)
    # print(N)
    print(U)
    # print(u)
    # print(v)
